In [ ]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools import copy_http_content

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps'
osprey = WPSClient(url)

In [ ]:
# NBVAL_IGNORE_OUTPUT
thunderbird.update_metadata?